In [12]:
import json
import pandas as pd
import re

In [13]:
# Path to the JSON file
file_path = '1716554847_127206(1).json'

# Read the file line by line and parse each JSON object
data = []
with open(file_path, 'r') as file:
    for line in file:
        data.append(json.loads(line))

In [14]:
# Extract the 'result' part of each JSON object and create a dataset
results = [entry['result'] for entry in data]

# Convert the list of results to a DataFrame
results_df = pd.DataFrame(results)

# Display the DataFrame
print(results_df.head())

                                                _raw  \
0  May 24 14:33:29 172.22.7.248 May 24 14:33:29 P...   
1  May 24 14:33:29 172.22.7.248 May 24 14:33:29 P...   

                           _time action   app  bytes bytes_in bytes_out  \
0  2024-05-24T14:33:29.000+02:00     OK  HTTP  41591       68     41523   
1  2024-05-24T14:33:29.000+02:00     OK  HTTP  26468       45     26423   

                                           category charset  \
0  application/x-www-form-urlencoded; charset=UTF-8   UTF-8   
1  application/x-www-form-urlencoded; charset=UTF-8   UTF-8   

                                         client_type  ... timestartpos  \
0  Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:1...  ...            0   
1  Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:1...  ...            0   

                   uri_path uri_query  \
0    /fr/graphique/affaires             
1  /fr/graphique/croissance             

                                                 url  \
0  "web4gfo.

In [15]:
# Define a function to extract variables from the _raw column
def extract_variables(raw_string):
    # Define the regular expression pattern with named groups
    pattern = re.compile(
        # Match timestamp (e.g., "May 24 14:33:29")
        r'(?P<timestamp>[A-Za-z]+\s+\d+\s+\d+:\d+:\d+)\s+'
        # Match host IP (e.g., "172.22.7.248")
        r'(?P<host_ip>\d+\.\d+\.\d+\.\d+)\s+'
        # Match any alphanumeric characters, dashes, or dots (e.g., "PD-HLB01-ID261.dc2.sia.priv")
        r'[A-Za-z0-9\-\.]+\s+'
        # Match the info level (e.g., "info")
        r'(?P<info>[a-z]+)\s+'
        # Match tmm with digits in brackets (e.g., "tmm[20235]")
        r'tmm\[\d+\]:\s+'
        # Match the rule description
        r'Rule\s+/Common/Irule-Syslog-splunk_v2\s+<HTTP_RESPONSE>:\s+'
        # Match src_ip (e.g., "src_ip="10.92.115.117%5"")
        r'src_ip="(?P<src_ip>[^"]+)",'
        # Match vip (e.g., "vip="10.210.248.11%5"")
        r'vip="(?P<vip>[^"]+)",'
        # Match http_method (e.g., "http_method="POST"")
        r'http_method="(?P<http_method>[^"]+)",'
        # Match http_host (e.g., "http_host="web4gfo.preprod.insee.fr:443"")
        r'http_host="(?P<http_host>[^"]+)",'
        # Match http_uri (e.g., "http_uri="/fr/graphique/affaires"")
        r'http_uri="(?P<http_uri>[^"]+)",'
        # Match http_url (e.g., "http_url="web4gfo.preprod.insee.fr:443/fr/graphique/affaires"")
        r'http_url="(?P<http_url>[^"]+)",'
        # Match http_version (e.g., "http_version="1.1"")
        r'http_version="(?P<http_version>[^"]+)",'
        # Match http_user_agent (e.g., "http_user_agent="Mozilla/5.0 ..."")
        r'http_user_agent="(?P<http_user_agent>[^"]+)",'
        # Match http_content_type (e.g., "http_content_type="application/x-www-form-urlencoded; charset=UTF-8"")
        r'http_content_type="(?P<http_content_type>[^"]+)",'
        # Match http_referrer (e.g., "http_referrer="https://web4gfo.preprod.insee.fr/fr/accueil"")
        r'http_referrer="(?P<http_referrer>[^"]+)",'
        # Match req_start_time (e.g., "req_start_time="2024/05/24 14:33:29"")
        r'req_start_time="(?P<req_start_time>[^"]+)",'
        # Match cookie (e.g., "cookie="JSESSIONID"")
        r'cookie="(?P<cookie>[^"]*)",'
        # Match user (e.g., "user=""")
        r'user="(?P<user>[^"]*)",'
        # Match virtual_server (e.g., "virtual_server="/Common/pool_ppweb4gfoln071 0"")
        r'virtual_server="(?P<virtual_server>[^"]+)",'
        # Match bytes_in (e.g., "bytes_in="68"")
        r'bytes_in="(?P<bytes_in>\d+)",'
        # Match res_start_time (e.g., "res_start_time="2024/05/24 14:33:29"")
        r'res_start_time="(?P<res_start_time>[^"]+)",'
        # Match node (e.g., "node="10.210.56.47%5"")
        r'node="(?P<node>[^"]+)",'
        # Match node_port (e.g., "node_port="80"")
        r'node_port="(?P<node_port>\d+)",'
        # Match http_status (e.g., "http_status="200"")
        r'http_status="(?P<http_status>\d+)",'
        # Match req_elapsed_time (e.g., "req_elapsed_time="193"")
        r'req_elapsed_time="(?P<req_elapsed_time>\d+)",'
        # Match bytes_out (e.g., "bytes_out="41523"")
        r'bytes_out="(?P<bytes_out>\d+)"'
    )
    match = pattern.match(raw_string)
    if match:
        return match.groupdict()
    return {}

In [16]:
# Apply the function to the _raw column and expand the dictionaries into columns
extracted_df = results_df['_raw'].apply(lambda x: pd.Series(extract_variables(x)))

# Concatenate the extracted columns with the original DataFrame
final_df = pd.concat([results_df, extracted_df], axis=1)

# Drop the original _raw column as it is no longer needed
final_df = final_df.drop(columns=['_raw'])

# Display the new columns
new_columns = extracted_df.columns.tolist()
print("New Columns: ", new_columns)

# Display the original columns
original_columns = results_df.columns.tolist()
print("Original Columns: ", original_columns)

# Display the DataFrame
print(final_df.head())

# Optionally, save the final DataFrame to a CSV file
final_df.to_csv('final_results_dataset.csv', index=False)

New Columns:  []
Original Columns:  ['_raw', '_time', 'action', 'app', 'bytes', 'bytes_in', 'bytes_out', 'category', 'charset', 'client_type', 'cookie', 'date_hour', 'date_mday', 'date_minute', 'date_month', 'date_second', 'date_wday', 'date_year', 'date_zone', 'dest', 'dest_addr', 'dest_ip', 'dest_port', 'dest_translated_ip', 'duration', 'eventtype', 'f5_bigip_server_host', 'host', 'http_content_type', 'http_host', 'http_method', 'http_referrer', 'http_referrer_domain', 'http_status', 'http_uri', 'http_url', 'http_user_agent', 'http_user_agent_length', 'http_version', 'index', 'is_f5_heartbeat', 'linecount', 'node', 'node_port', 'pool', 'protocol', 'punct', 'req_elapsed_time', 'req_start_time', 'res_start_time', 'response_code', 'rtt', 'site', 'source', 'sourcetype', 'splunk_server', 'src', 'src_addr', 'src_ip', 'status', 'tag', 'tag::eventtype', 'thruput', 'timeendpos', 'timestartpos', 'uri_path', 'uri_query', 'url', 'url_domain', 'url_length', 'user', 'vendor_product', 'vip', 'virtu